3D энкодер- декодер. Отличие от предыдущей версии - вместо растра точек на вход сети подать вектора лендмарков в виде набора координат и длин.
к MDWL добавлены еще 2 лендмарка BCP, FAP - размер энкодера вырос на 3 +3 значения итого 12 значений* 16 зубов = 192.

In [11]:
# from keras.datasets import mnist
import numpy as np
# from jaw_gen import *
import tensorflow 
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import tensorflow.keras.backend as K
from tensorflow.keras.layers import Lambda
import cv2

from tqdm import tqdm
# основное отличие - Landmark_gen больше не генерит картинки. только вектора. 
# и у него теперь есть 3D рисовалка, которая есть независимый метод
# генератор теперь в другом файле, генерит дополнительно 2 лендмарка
from jaw_gen3d_3lm import Landmark_gen 
from csv_parser_pd import set_gen_fr_csv_pd

# готовим СИНТЕТИЧЕСКИЕ входные данные 

In [12]:

SYNT = 0                     # True - учим на синтетике, False - на реальных

# готовим СИНТЕТИЧЕСКИЕ входные данные 
inst_ =                 Landmark_gen()
data_len =              5000
dataset =               []
dataset_spoiled =       []
dataset_vec =           []
dataset_vec_spoiled =   []

if SYNT:
    for i in range(data_len):
        scale =  9 + np.random.sample()*3     # это для размера 200, диапазон от 14 до 19 для размера 400
        factor = 2.2 + np.random.sample()/4   # это для размера 200, диапазон 2.5 - 2.7 для размера 400 
        # name = f"scale {scale:.4}  factor {factor:.3}"
        # сдвиги и генерация остальных лендмарков по умолчанию включены. 
        vec, vec_spoiled = inst_.image_gen(scale=scale, factor=factor, spoiledMDW=True, shiftMDW=True)

        # собираем датасет из векторов 
        dataset_vec.append(vec)
        dataset_vec_spoiled.append(vec_spoiled)
    # print (f"dataset_vec {dataset_vec[:1]}")
    print (f"dataset_vec len {len(dataset_vec)} ")
    print (f"dataset_vec_spoiled len {len(dataset_vec_spoiled)} ")

    # преобразуем датасет в numpy массив
    dataset_vec =           np.array(dataset_vec,           dtype="float32") # /2550 тогда не учится ничего 
    dataset_vec_spoiled =   np.array(dataset_vec_spoiled,   dtype="float32") # /2550

# print (f"datraset vec {dataset_vec[:1]}")


# готовим данные из файла CSV

In [13]:
# готовим датасет из файла csv
# csv_fpath = 'C:/Users/Anton/Projects/jaw_encoder/csv/input_004.csv'
# csv_fpath = 'C:/Users/Anton/Projects/jaw_encoder/csv/test10.csv'
csv_fpath = 'C:/Users/Anton/Projects/jaw_encoder/csv/input.csv'

# выбираем данные из файла
if not SYNT:
    dataset_vec_spoiled, dataset_vec = set_gen_fr_csv_pd(csv_fpath) # возвращает np массивы сначала T0

data_len = len(dataset_vec)
print (f"data lenght {data_len}")
# [print (f"{i}val {dataset_vec[i][5]} \n {i}spoil {dataset_vec_spoiled[i][5]}") for i in range(len(dataset_vec))]
# print (f"val {dataset_vec[3][5]} \n {dataset_vec_spoiled[3][5]}") 

data lenght 2398


In [14]:
# разобьем датасет на тренировочную и тестовую части
set_divider_ = round(data_len*0.9)

# векторный датасет
x_train_vec =   dataset_vec[:set_divider_]
x_test_vec =    dataset_vec[set_divider_:]
# print (f"length vec {len(x_train_vec)} - {len(x_test_vec)}")

x_train_vec_spoiled =   dataset_vec_spoiled[:set_divider_]
x_test_vec_spoiled =    dataset_vec_spoiled[set_divider_:]
# print (f"vec [0]{x_test_vec[0]}")
print (f"length vec         {len(x_train_vec)} - {len(x_test_vec)}")
print (f"length vec spoil   {len(x_train_vec_spoiled)} - {len(x_test_vec_spoiled)}")

length vec         2158 - 240
length vec spoil   2158 - 240


In [15]:
# Размерности слоев энкодера 
dense_dim = 192             # размерность полносвязного слоя энкодера 16 зубов * 12 значений.
dim_code = dense_dim//6     # размерность кодированного слоя

In [16]:
WITH_BN = 0 # go with batcNormalization

def bn(x):
    return BatchNormalization()(x) if WITH_BN else x

def create_vec_ae3D():
    # 3d кодер - декодер содержит на входе 12*16=192 значений.
    activation = 'elu' 
    # activation = 'linear'
    inputs = Input(dense_dim,)

    # encoder
    x = Dense(dense_dim, activation=activation)(inputs)
    x = bn(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = bn(x)
    code = Dense(dim_code, activation='linear')(x)

    # decoder
    input_encoded = Input((dim_code,))
    x = Dense(dim_code, activation=activation)(input_encoded)
    x = bn(x)
    x = Dense(dense_dim, activation=activation)(x)
    x = bn(x)
    out = Dense(dense_dim, activation='linear')(x)
    
    # Модели
    encoder = Model(inputs, code, name="encoder")
    decoder = Model(input_encoded, out, name="decoder")
    autoencoder = Model(inputs, decoder(encoder(inputs)), name="autoencoder")
    return encoder, decoder, autoencoder

encoder, decoder, autoencoder = create_vec_ae3D()
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
# autoencoder.compile(Adam(0.0001), loss='mean_squared_error') 

# c_autoencoder.summary()

In [17]:
batch_size = 128
def create_denoising_model(autoencoder):
    
    inputs  = Input(batch_shape=(batch_size, dense_dim))
    inputs_spoiled  = Input(batch_shape=(batch_size, dense_dim)) # вход для кореженных зубьев

    denoiser_model = Model([inputs, inputs_spoiled], autoencoder(inputs_spoiled), name="denoiser")
    # return noiser, denoiser_model
    return denoiser_model


# noiser, denoiser_model = create_denoising_model(c_autoencoder)
denoiser_model = create_denoising_model(autoencoder)
denoiser_model.compile(optimizer='adam', loss='mean_squared_error')
# denoiser_model.compile(Adam(1e-4), loss='mean_squared_error')
# optimizer=tf.keras.optimizer.Adam(learning_rate=1e-3)
# keras.optimizers.Adam(learning_rate=1e-3)

In [18]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

def plot_digits(*args):
    # for a in args:
        # print (f"type args {type(a)}")
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    
    plt.figure(figsize=(2*n, 2*len(args)))
    for j in range(n):
            for i in range(len(args)):
                ax = plt.subplot(len(args), n, i*n + j + 1)
                plt.imshow(args[i][j])
                plt.gray()
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

    plt.show()

In [19]:
# вместо фита будем делать  model.train_on_batch читобы смотреть как учится и куда движемся 
# хотя у фита много больше возможностей с колбеками
def train_on_batch(vec, vec_spoil, batch_size=batch_size, epochs=100):
    # подровняем массив картинок и обучающих векторов так, чтоб np.reshape не ругался - на несовпадение размерности
    # сделаем его длину кратной размеру батча
    
    vec =   vec [:len(x_train_vec)//batch_size*batch_size]
    vec =   np.reshape(vec, (-1, batch_size, dense_dim))           # x_test  = np.reshape(x_test,  (len(x_test),  200, 200, 1))
    
    # и массив кореженных тоже
    vec_spoil = vec_spoil   [:len(x_train_vec_spoiled)  //batch_size*batch_size]
    vec_spoil = np.reshape(vec_spoil,   (-1, batch_size, dense_dim)) 

    min_loss = 10e100 # потери для сравнения эпох будем тут сохранять 

    for epoch in range(1, epochs+1):
        # print(f"Epoch {epoch}")
        for i, _ in enumerate(vec):
            batch_vec =         vec[i] 
            batch_vec_spoiled = vec_spoil[i]

            loss = denoiser_model.train_on_batch([batch_vec, batch_vec_spoiled], batch_vec)

        # эпоха отучилась, построим что вышло.
        n = 10
        vecs = batch_vec                    # это последний батч в каждой эпохе.
        # print (f"1! vecs shape{vecs.shape}") # shape - (256, 64)
        noised_vecs = batch_vec_spoiled     
        # а картинки не участвуют в учении, количество бачей у них при делении датасета на бачи
        # с векторами разное, поэтому найти соответствующую картинку проблематично. 
        # noised_imgs = noiser.predict(imgs, batch_size=batch_size)

        # задача в том, чтобы картинке сопоставить спойленую картинку ( это не сложно из сгенерированной 
        # выборки сделать), но потом предиктить для этой картинки вектора. и рисовать их на новой картинке 
        # средствами opencv 
        encoded_vecs = encoder.predict(noised_vecs[:n],  batch_size=n)
        decoded_vecs = decoder.predict(encoded_vecs[:n], batch_size=n)

        # формируем картинки из предсказанных векторов
        vecs =              np.reshape(vecs,            (-1, 16, dense_dim//16))
        noised_vecs =       np.reshape(noised_vecs,     (-1, 16, dense_dim//16))
        predicted_vecs =    np.reshape(decoded_vecs,    (-1, 16, dense_dim//16))
        
        # print (f"vecs shape{vecs.shape} vecs[0] {vecs[0]}")
        if (epoch%100 ==0) or (epoch in [1,2, 5, 10, 30, 60]):
            print(f"Epoch {epoch} loss -{loss}")
        # рисуем картинки и сохраняем модели
        if (epoch % 50 ==0) and (epoch >= 5000):
            # print(f'Epoch {epoch}')
            # plot_digits(batch_x[:n], batch_x_spoil[:n], imgs_fr_vec[:n], imgs_spoil_fr_vec[:n], imgs_fr_decoded_vec[:n])            
            if loss < min_loss:
                min_loss = loss
                # save only the best models
                #denoiser_model.save(f'models3D/denoiser{epoch}ep_loss__{loss:.3}__.h5')
                decoder.save(f'models3D/decoder{epoch}ep_{data_len}len_{batch_size}btsz_bn{WITH_BN}__loss__{loss:.3}__.h5')
                encoder.save(f'models3D/encoder{epoch}ep_{data_len}len_{batch_size}btsz_bn{WITH_BN}__loss__{loss:.3}__.h5')
        
train_on_batch(x_train_vec, x_train_vec_spoiled, batch_size=batch_size, epochs=20000)

Epoch 1 loss -279.517578125
Epoch 2 loss -261.01715087890625
Epoch 5 loss -145.24343872070312
Epoch 10 loss -26.741870880126953
Epoch 30 loss -14.591438293457031
Epoch 60 loss -13.965314865112305
Epoch 100 loss -13.56088924407959
Epoch 200 loss -13.23515510559082
Epoch 300 loss -13.255166053771973
Epoch 400 loss -13.03404426574707
Epoch 500 loss -13.097204208374023
Epoch 600 loss -12.943670272827148
Epoch 700 loss -12.9944429397583
Epoch 800 loss -12.906669616699219
Epoch 900 loss -12.893836975097656
Epoch 1000 loss -12.856348037719727
Epoch 1100 loss -12.866344451904297
Epoch 1200 loss -12.842496871948242
Epoch 1300 loss -12.817785263061523
Epoch 1400 loss -12.756834983825684
Epoch 1500 loss -12.795297622680664
Epoch 1600 loss -12.774004936218262
Epoch 1700 loss -12.719472885131836
Epoch 1800 loss -12.76976203918457
Epoch 1900 loss -12.784340858459473
Epoch 2000 loss -12.72946548461914
Epoch 2100 loss -12.716228485107422
Epoch 2200 loss -12.731800079345703
Epoch 2300 loss -12.68888473

In [20]:
# c_autoencoder.save(f'models/c_autoencoder.h5'